In [65]:
import glob
import json

import pandas as pd

# Get all json files in benchmark folder
json_files = glob.glob("rag-qa-benchmarking/results_gpt-4o-2024-11-20/*.json")

# List to store data from each file
data = []

# Read each json file
for file in json_files:
    with open(file) as f:
        json_data = json.load(f)
        # Extract qid from filename
        qid = file.split("/")[-1].replace(".json", "")
        json_data["qid"] = qid
        data.append(json_data)

# Create dataframe
df = pd.DataFrame(data).set_index("qid")


In [66]:
df["llm_thinking"] = df["llm_response"].str.extract(r"<thinking>(.*?)</thinking>", expand=False).str.strip()
df = df.drop("llm_response", axis=1)

In [67]:
print("Total questions:", df.shape[0])
print("Proportion of questions where PaperQA won:", df["winner"].value_counts(normalize=True)["paperqa"], "total:", df["winner"].value_counts()["paperqa"])
# print("Proportion of questions where there was a tie:", df['winner'].value_counts(normalize=True)['tie'], "total:", df['winner'].value_counts()['tie'])
# print("Proportion of questions where human won:", df['winner'].value_counts(normalize=True)['human'], "total:", df['winner'].value_counts()['human'])
print()

df_human_winner = df[df["winner"] == "human"]
for idx, row in df_human_winner.iterrows():
    print("=" * 80)
    print(f"Question: {row['question']}")
    print(f"\nPaperQA Answer:\n{row['paperqa_answer']}")
    print(f"\nHuman Answer:\n{row['human_answer']}")
    print(f"\nLLM Response:\n{row['llm_thinking']}")
    print("\n")


Total questions: 20
Proportion of questions where PaperQA won: 0.95 total: 19

Question: what is the charge of n type semiconductor?

PaperQA Answer:
An n-type semiconductor is primarily negatively charged due to the abundance of free electrons, which are the majority charge carriers. This negative charge arises from the substitution of a Group 14 atom in the crystal lattice with a Group 15 atom, introducing extra electrons into the conduction band (1519498 chunk 1). The overall charge of the n-type semiconductor is determined by these free electrons, making it negatively charged .

However, when an n-type semiconductor is paired with a p-type semiconductor to form a p-n junction, the region near the junction on the n-type side develops a layer of positive charge carriers. This occurs due to the movement of electrons from the n-type material into the p-type material, where they fill holes. Despite this localized positive charge near the junction, the overall charge of the n-type semico

In [68]:
df["were_gt_docs_found"].value_counts(normalize=True)

were_gt_docs_found
False    0.55
True     0.45
Name: proportion, dtype: float64

In [69]:
docs = pd.read_csv(
    "rag-qa-benchmarking/science_test_collection.tsv",
    sep="\t",
    header=None,
)
docs.columns = ["doc_id", "doc_text"]

In [71]:
row = df.iloc[5]
print("Was GT docs found?", row["were_gt_docs_found"])
print(f"Question: {row['question']}")
print("=" * 80)

complete_human_answer = ""
complete_human_answer += "References\n\n"
for _id in row["gt_doc_ids"]:
    complete_human_answer += f"Document {_id}: {docs[docs['doc_id'] == _id]['doc_text'].values[0]}\n"

complete_human_answer += "\nFinal Answer\n"
complete_human_answer += "\n" + row["human_answer"]

print(complete_human_answer)

print("=" * 80)

complete_pqa_answer = ""
complete_pqa_answer += "References\n\n"
for _id in row["paper_search_ids"]:
    complete_pqa_answer += f"Document {_id}: {docs[docs['doc_id'] == _id]['doc_text'].values[0]}\n"

complete_pqa_answer += "\nFinal Answer\n"
complete_pqa_answer += "\n" + row["paperqa_answer"]

print(complete_pqa_answer)

Was GT docs found? False
Question: a body is moving with a constant velocity?
References

Document 1509624: The zero force related to zero acceleration is not a property of the object, it is a statement about the forces acting on the body. That is your title should not read "has no force" but "is subject to no net force". If a body has a non-zero, but constant, velocity then we know that the total of all the forces applied to it is zero (from Newton's Laws). We also know that is has non-zero "momentum", and when it collides with another object some (or all) of that momentum can be transferred to the other object. During the collision the body is subjected to new forces and the net force is no longer zero meaning that it will accelerate.

Final Answer

If a body has a non-zero, but constant, velocity then we know from Newton's Law that the total of all the forces applied to it is zero. 
We also know that it has non-zero momentum, and when it collides with another object, some (or all) o